In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [16]:
fake_transaction = pd.read_csv('simulated_transaction_2024.csv')

In [17]:
fake_transaction

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
0,01/01/2023,00:00,678330503.0,2971.000000,1584.00,NaN,Westport Care Home
1,01/01/2023,00:00,472213568.0,3792.000000,1950.00,NaN,Barbiee Boutique
2,01/01/2023,00:00,472213568.0,3012.000000,-780.00,283027736.0,NaN
3,01/01/2023,00:00,283027736.0,1787.000000,780.00,472213568.0,NaN
4,01/01/2023,00:00,624500124.0,3226.000000,1825.00,NaN,Fat Face
...,...,...,...,...,...,...,...
230591,06/12/2023,20:54,581655972.0,45935.206861,-41.06,NaN,Tesco
230592,06/12/2023,20:55,786141370.0,-244.837500,-62.35,NaN,Sainsbury Local
230593,06/12/2023,21:05,824916823.0,9709.172159,-32.94,NaN,Deliveroo
230594,06/12/2023,21:13,366550080.0,26834.165794,-19.25,NaN,Amazon


In [18]:
fake_transaction.columns

Index(['Date', 'Timestamp', 'Account No', 'Balance', 'Amount',
       'Third Party Account No', 'Third Party Name'],
      dtype='object')

In [19]:
fake_transaction = fake_transaction[['Date', 'Timestamp', 'Account No', 'Balance', 'Amount', 'Third Party Name']]

In [20]:
fake_transaction

,Date,Timestamp,Account No,Balance,Amount,Third Party Name
0,01/01/2023,00:00,678330503.0,2971.000000,1584.00,Westport Care Home
1,01/01/2023,00:00,472213568.0,3792.000000,1950.00,Barbiee Boutique
2,01/01/2023,00:00,472213568.0,3012.000000,-780.00,NaN
3,01/01/2023,00:00,283027736.0,1787.000000,780.00,NaN
4,01/01/2023,00:00,624500124.0,3226.000000,1825.00,Fat Face
...,...,...,...,...,...,...
230591,06/12/2023,20:54,581655972.0,45935.206861,-41.06,Tesco
230592,06/12/2023,20:55,786141370.0,-244.837500,-62.35,Sainsbury Local
230593,06/12/2023,21:05,824916823.0,9709.172159,-32.94,Deliveroo
230594,06/12/2023,21:13,366550080.0,26834.165794,-19.25,Amazon


In [21]:
fake_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230596 entries, 0 to 230595
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Date              230367 non-null  object 
 1   Timestamp         230345 non-null  object 
 2   Account No        230372 non-null  float64
 3   Balance           230350 non-null  float64
 4   Amount            230387 non-null  float64
 5   Third Party Name  223517 non-null  object 
dtypes: float64(3), object(3)
memory usage: 10.6+ MB


In [22]:
fake_transaction.isnull().sum()

Date                 229
Timestamp            251
Account No           224
Balance              246
Amount               209
Third Party Name    7079
dtype: int64

In [23]:
fake_transaction.dropna(subset=['Third Party Name'], inplace=True)

/var/folders/j7/k1zn5v217flb70ktdfw_fzdc0000gn/T/ipykernel_32811/653314486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_transaction.dropna(subset=['Third Party Name'], inplace=True)


In [24]:
fake_transaction['Date'] = pd.to_datetime(fake_transaction['Date'], format='%d/%m/%Y')
fake_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 223517 entries, 0 to 230595
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              223293 non-null  datetime64[ns]
 1   Timestamp         223271 non-null  object        
 2   Account No        223297 non-null  float64       
 3   Balance           223277 non-null  float64       
 4   Amount            223317 non-null  float64       
 5   Third Party Name  223517 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 11.9+ MB


/var/folders/j7/k1zn5v217flb70ktdfw_fzdc0000gn/T/ipykernel_32811/2944397103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_transaction['Date'] = pd.to_datetime(fake_transaction['Date'], format='%d/%m/%Y')


In [25]:
fake_transaction.isnull().sum()

Date                224
Timestamp           246
Account No          220
Balance             240
Amount              200
Third Party Name      0
dtype: int64

In [26]:
print(fake_transaction['Amount'].mean())
print(fake_transaction['Amount'].median())
print(fake_transaction['Amount'].mode()[0])

6.879775700013879
-28.69
-18.99


In [27]:
median_amount = fake_transaction['Amount'].median()

fake_transaction['Amount'] = fake_transaction['Amount'].fillna(median_amount)

fake_transaction

/var/folders/j7/k1zn5v217flb70ktdfw_fzdc0000gn/T/ipykernel_32811/447577542.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_transaction['Amount'] = fake_transaction['Amount'].fillna(median_amount)


,Date,Timestamp,Account No,Balance,Amount,Third Party Name
0,2023-01-01,00:00,678330503.0,2971.000000,1584.00,Westport Care Home
1,2023-01-01,00:00,472213568.0,3792.000000,1950.00,Barbiee Boutique
4,2023-01-01,00:00,624500124.0,3226.000000,1825.00,Fat Face
5,2023-01-01,00:00,203466392.0,4607.660000,2841.66,Lavender Primary
6,2023-01-01,00:00,768271776.0,3620.000000,1950.00,A Cut Above
...,...,...,...,...,...,...
230591,2023-12-06,20:54,581655972.0,45935.206861,-41.06,Tesco
230592,2023-12-06,20:55,786141370.0,-244.837500,-62.35,Sainsbury Local
230593,2023-12-06,21:05,824916823.0,9709.172159,-32.94,Deliveroo
230594,2023-12-06,21:13,366550080.0,26834.165794,-19.25,Amazon


In [28]:
fake_transaction.isnull().sum()

Date                224
Timestamp           246
Account No          220
Balance             240
Amount                0
Third Party Name      0
dtype: int64

In [29]:
print(fake_transaction['Balance'].mean())
print(fake_transaction['Balance'].median())
print(fake_transaction['Balance'].mode()[0])

3676.6946254832383
1736.950800000002
10.0


In [30]:
median_balance = fake_transaction['Balance'].median()

fake_transaction['Balance'] = fake_transaction['Balance'].fillna(median_balance)

fake_transaction.isnull().sum()

/var/folders/j7/k1zn5v217flb70ktdfw_fzdc0000gn/T/ipykernel_32811/1176191090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_transaction['Balance'] = fake_transaction['Balance'].fillna(median_balance)


Date                224
Timestamp           246
Account No          220
Balance               0
Amount                0
Third Party Name      0
dtype: int64

In [31]:
fake_transaction

,Date,Timestamp,Account No,Balance,Amount,Third Party Name
0,2023-01-01,00:00,678330503.0,2971.000000,1584.00,Westport Care Home
1,2023-01-01,00:00,472213568.0,3792.000000,1950.00,Barbiee Boutique
4,2023-01-01,00:00,624500124.0,3226.000000,1825.00,Fat Face
5,2023-01-01,00:00,203466392.0,4607.660000,2841.66,Lavender Primary
6,2023-01-01,00:00,768271776.0,3620.000000,1950.00,A Cut Above
...,...,...,...,...,...,...
230591,2023-12-06,20:54,581655972.0,45935.206861,-41.06,Tesco
230592,2023-12-06,20:55,786141370.0,-244.837500,-62.35,Sainsbury Local
230593,2023-12-06,21:05,824916823.0,9709.172159,-32.94,Deliveroo
230594,2023-12-06,21:13,366550080.0,26834.165794,-19.25,Amazon
